In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
144,This is a fabulous film.<br /><br /> The plot ...,positive
611,"Getting lost in space frozen for 15 years, tha...",negative
431,I read a couple of good reviews on this board ...,negative
39,I am a big fan of Fred MacMurray and Carole Lo...,negative
381,I've read the other reviews and found some to ...,positive


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
144,fabulous film br br plot good yarn imaginative...,positive
611,getting lost space frozen year that s unlikely...,negative
431,read couple good review board mr scarface anyo...,negative
39,big fan fred macmurray carole lombard and addi...,negative
381,read review found comparison movie v real life...,positive


In [9]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
144,fabulous film br br plot good yarn imaginative...,1
611,getting lost space frozen year that s unlikely...,0
431,read couple good review board mr scarface anyo...,0
39,big fan fred macmurray carole lombard and addi...,0
381,read review found comparison movie v real life...,1


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/TalkeenAhmadNomani/capstone_project.mlflow')
dagshub.init(repo_owner='TalkeenAhmadNomani', repo_name='capstone_project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "TalkeenAhmadNomani/capstone_project"

Repository TalkeenAhmadNomani/capstone_project initialized!

2025/05/25 00:18:17 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/af31794ce9884c5cbe6d4a298f7aa08f', creation_time=1748112496678, experiment_id='0', last_update_time=1748112496678, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-25 00:18:56,251 - INFO - Starting MLflow run...
2025-05-25 00:18:56,831 - INFO - Logging preprocessing parameters...
2025-05-25 00:18:57,940 - INFO - Initializing Logistic Regression model...
2025-05-25 00:18:57,940 - INFO - Fitting the model...
2025-05-25 00:18:57,970 - INFO - Model training complete.
2025-05-25 00:18:57,970 - INFO - Logging model parameters...
2025-05-25 00:18:58,335 - INFO - Making predictions...
2025-05-25 00:18:58,337 - INFO - Calculating evaluation metrics...
2025-05-25 00:18:58,356 - INFO - Logging evaluation metrics...
2025-05-25 00:18:59,921 - INFO - Saving and logging the model...
2025/05/25 00:19:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-25 00:19:11,145 - INFO - Model training and logging completed in 14.31 seconds.
2025-05-25 00:19:11,146 - INFO - Accuracy: 0.736
2025-05-25 00:19:11,147 - INFO - Precision: 

🏃 View run chill-fawn-863 at: https://dagshub.com/TalkeenAhmadNomani/capstone_project.mlflow/#/experiments/0/runs/637ed971cf9347d7b207eb83357af5bc
🧪 View experiment at: https://dagshub.com/TalkeenAhmadNomani/capstone_project.mlflow/#/experiments/0
